In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [7]:
import pandas as pd
pd.read_csv('nyc_restaurant_inspections.csv').head(2)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE
0,50015075,NEW YORK FRIED CHICKEN,BROOKLYN,102,SARATOGA AVE,11233.0,7185522345,Chicken,04/28/2015,Violations were cited in the following area(s).,02G,Cold food item held above 41Âº F (smoked fish ...,Critical,16.0,B,04/28/2015,03/01/2018,Cycle Inspection / Re-inspection
1,40364529,BEN-BEST DELI & RESTAURANT,QUEENS,9640,QUEENS BOULEVARD,11374.0,7188971700,Jewish/Kosher,03/24/2016,Violations were cited in the following area(s).,05B,"Harmful, noxious gas or vapor detected. CO ~1 ...",Critical,42.0,C,03/24/2016,03/01/2018,Cycle Inspection / Re-inspection


In [2]:
inspections = sc.textFile('nyc_restaurant_inspections.csv')

In [4]:
list(enumerate(inspections.first().split(',')))

[(0, 'CAMIS'),
 (1, 'DBA'),
 (2, 'BORO'),
 (3, 'BUILDING'),
 (4, 'STREET'),
 (5, 'ZIPCODE'),
 (6, 'PHONE'),
 (7, 'CUISINE DESCRIPTION'),
 (8, 'INSPECTION DATE'),
 (9, 'ACTION'),
 (10, 'VIOLATION CODE'),
 (11, 'VIOLATION DESCRIPTION'),
 (12, 'CRITICAL FLAG'),
 (13, 'SCORE'),
 (14, 'GRADE'),
 (15, 'GRADE DATE'),
 (16, 'RECORD DATE'),
 (17, 'INSPECTION TYPE')]

In [17]:
inspections.flatMap(lambda line: line.split()) \
        .map(lambda x: (x.lower(), 1)) \
        .reduceByKey(lambda x,y: x+y).take(5)

[('code,violation', 1),
 ('description,critical', 1),
 ('date,inspection', 1),
 ('following', 394853),
 ('area(s).,02g,cold', 23563)]

In [15]:
def extractDesc(partId, records):
    if partId==0:
        next(records)
    import csv
    reader = csv.reader(records)
    for row in reader:
        (camis,desc) = (int(row[0]), row[7])
        yield (camis, desc)

desc = inspections.mapPartitionsWithIndex(extractDesc)
desc.take(5)

[(50015075, 'Chicken'),
 (40364529, 'Jewish/Kosher'),
 (41580756, 'Caribbean'),
 (50001255, 'Italian'),
 (50016437, 'Korean')]

In [46]:
desc.map(lambda x: (x[1].lower(),1)).reduceByKey(lambda x,y: x+y).take(5)

[('sandwiches/salads/mixed buffet', 2762),
 ('american', 84995),
 ('juice, smoothies, fruit salads', 3343),
 ('tex-mex', 2091),
 ('latin (cuban, dominican, puerto rican, south & central american)', 17400)]

In [50]:
camis = desc.map(lambda x: (x[0],1)).reduceByKey(lambda x,y: x+y)
camis.take(5)

[(50015075, 12),
 (50001255, 13),
 (50015315, 50),
 (50008170, 30),
 (50000045, 29)]

In [44]:
thing = desc.join(camis).values() \
    .reduceByKey(lambda x,y: (x[0] + y[0],x[1]+y[1])) \
    .mapvalues(lambda x: int(x[0]/x[1])) \
    .collect()

AttributeError: 'PipelinedRDD' object has no attribute 'mapvalues'